In [ ]:
import pdfplumber
import pandas as pd

pdf_path = "/Users/Dan/Desktop/Hedge Fund Dash/linkusdt 45min - DaviddTech.pdf"
all_tables = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        print(f"🔍 Checking page {i+1}")
        tables = page.extract_tables()
        if tables:
            for table in tables:
                df = pd.DataFrame(table)
                # Check if it's the trade table by column headers
                if df.shape[1] >= 5 and df.iloc[0][0] and "Trade" in df.iloc[0][0]:
                    print(f"✅ Table detected on page {i+1}")
                    df.columns = df.iloc[0]  # Set first row as header
                    df = df[1:]  # Remove header from data
                    all_tables.append(df)

if not all_tables:
    print("⚠️ No trade tables found in the PDF.")
else:
    full_table = pd.concat(all_tables, ignore_index=True)
    full_table = full_table.dropna(how='all')  # Drop empty rows
    print("📊 Extracted table preview:")
    print(full_table.head())

    # Optional: save to CSV for review
    full_table.to_csv("extracted_trades.csv", index=False)


In [ ]:
import pandas as pd

# Load the Airtable CSV
csv_path = "/Users/Dan/Desktop/Hedge Fund Dash/Forward Testing Results/Strategies-👁️ Watch List.csv"
df = pd.read_csv(csv_path)

# Clean column names
df.columns = [col.strip() for col in df.columns]

# === Begin Transformations ===

# 1. Status: Live trades if value >= 1
df["status"] = df["Live Trades"].apply(lambda x: "Live trades" if x >= 1 else "Backtest")

# 2. Level based on TIMEFRAME
def map_level(tf):
    tf = str(tf).lower().strip()
    if any(x in tf for x in ["1", "3", "5", "15"]) and "min" in tf:
        return "Aggressive"
    elif any(x in tf for x in ["30", "45"]) and "min" in tf:
        return "Moderate"
    elif "1h" in tf or "60" in tf:
        return "Conservative"
    else:
        return "Unknown"

df["Level"] = df["TIMEFRAME"].apply(map_level)

# 3. Instrument
df["instrument"] = df["COIN PAIR"]

# 4. Journal – placeholder
df["journal"] = ""

# 5. Settings = TIMEFRAME
df["Settings"] = df["TIMEFRAME"]

# 6. P&L – placeholder
df["P&L"] = ""

# 7. Win Rate
df["Win Rate"] = df["WIN RATE - LIVE"]

# 8. Total Trades
df["Total Trades"] = df["TRADES - LIVE"]

# 9. Winning Trades
df["Winning Trades"] = df["NUMBER WINNING TRADES"]

# 10. PF
df["PF"] = df["PROFIT FACTOR - LIVE"]

# 11. Net Profit $
df["Net Profit $"] = df["NET PROFIT % - LIVE"]

# 12. Years
df["FIRST TRADED DATE"] = pd.to_datetime(df["FIRST TRADED DATE"], errors="coerce")
df["LAST TRADED DATE"] = pd.to_datetime(df["LAST TRADED DATE"], errors="coerce")
df["Years"] = (df["LAST TRADED DATE"] - df["FIRST TRADED DATE"]).dt.days / 365

# 13. Avg Annual Return = Net Profit / Years
df["Avg Annual Return"] = df["Net Profit $"] / df["Years"]

# 14. Max Drawdown %
df["Max Drawdown%"] = df["MAX DRAWDOWN - LIVE"]

# 15. Max Drawdown $ – placeholder
df["Max Drawdown$"] = ""

# 16. Reward-to-Risk Ratio
df["Reward-to-Risk Ratio"] = df["Avg Annual Return"] / df["Max Drawdown%"]

# 17. RMD > 2
df["RMD > 2"] = df["Reward-to-Risk Ratio"].apply(lambda x: "Yes" if x > 2 else "No")

# 18. IMG – placeholder
df["IMG"] = ""

# Reorder the final columns for dashboard
final_columns = [
    "instrument", "journal", "status", "Level", "Settings", "P&L", "Win Rate", "Total Trades",
    "Winning Trades", "PF", "Net Profit $", "Years", "Avg Annual Return",
    "Max Drawdown%", "Max Drawdown$", "Reward-to-Risk Ratio", "RMD > 2", "IMG"
]

df_final = df[final_columns]

# Save to CSV
df_final.to_csv(output_path, index=False)

output_path


OSError: Cannot save file into a non-existent directory: '/mnt/data'